In [2]:
# ETL output_steam_games.json

# Importación de librerías
import pandas as pd
import datetime
import math

# Obtener datos del archivo json
games = pd.read_json('../data_sources/json/output_steam_games.json', lines=True)

# Eliminar las filas nulas
games = games.dropna(how="all").reset_index(drop=True)

# Eliminar las columnas innecesarias
games = games.drop(columns=['publisher', 'url', 'app_name', 'tags', 'reviews_url', 'specs', 'price', 'early_access', 'developer'])

# Buscar y eliminar filas duplicadas
games['genres'] = games['genres'].apply(lambda x: tuple(x) if isinstance(x, list) else x)
duplicates = games[games.duplicated(keep='first', subset=['id'])]
duplicates = duplicates.index.tolist()
games = games.drop(duplicates).reset_index(drop=True)

# Crear una nueva columna con el año de lanzamiento y eliminar la columna de fecha de lanzamiento
games['release_year'] = games['release_date'].str.extract(r'(\d{4})').fillna(0).astype(int)
games = games.drop(columns=['release_date'])

# Elimina las filas donde el campo title sea nulo
games = games.dropna(subset=['title'])

# Valida que el año de lanzamiento no sea mayor al año actual
current_year = datetime.datetime.now().year
games['release_year'] = games['release_year'].apply(lambda x: x if x <= current_year else 0)

# Dividir el DataFrame en bloques de 1000 filas
rows_per_file = 10000
total_rows = len(games)
total_files = math.ceil(total_rows / rows_per_file)

# Recorrer y crear archivos Parquet
for i in range(total_files):
    s = i * rows_per_file
    e = min((i + 1) * rows_per_file, total_rows)
    block = games.iloc[s:e]

    # Crear el nombre del archivo Parquet (puedes modificar el nombre como desees)
    file_name = f"../data_sources/parquet/steam_games/dataset_{i + 1}.parquet"

    # Guardar el bloque actual en un archivo Parquet
    block.to_parquet(file_name)

    print(f"Archivo {file_name} guardado con éxito. Filas {s + 1} a {e} del DataFrame.")

Archivo ../data_sources/parquet/steam_games/dataset_1.parquet guardado con éxito. Filas 1 a 10000 del DataFrame.
Archivo ../data_sources/parquet/steam_games/dataset_2.parquet guardado con éxito. Filas 10001 a 20000 del DataFrame.
Archivo ../data_sources/parquet/steam_games/dataset_3.parquet guardado con éxito. Filas 20001 a 30000 del DataFrame.
Archivo ../data_sources/parquet/steam_games/dataset_4.parquet guardado con éxito. Filas 30001 a 30083 del DataFrame.
